In [ ]:
#!g1.1
import pandas as pd
import numpy as np

df = pd.read_csv("/home/jupyter/datasphere/project/CW_dataset/CW_dataset.csv")



In [79]:
#!g1.1

# split the text into word lists for training set
list_of_texts = []
for i in range(df.shape[0]):
    #pull the words
    words = str(df['text'][i])
    #remove the punctuation
    q = len(df['text'][i].split())
    list_of_texts.append(q)
   


In [80]:
#!g1.1
import statistics
print(statistics.mean(list_of_texts))

567.4438295517432


In [26]:
#!g1.1
import transformers
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')






In [27]:
#!g1.1
import pandas as pd
import numpy as np
from datasets import load_dataset

dataset_train = load_dataset('/home/jupyter/datasphere/project/CW_dataset/', data_files='out.csv', split='train[:80%]')
dataset_test = load_dataset('/home/jupyter/datasphere/project/CW_dataset/', data_files='out.csv', split='train[80%:]')



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 149.75it/s]

Generating train split: 0 examples [00:00, ? examples/s]
Generating train split: 10000 examples [00:00, 50720.84 examples/s]


In [28]:
#!g1.1
def preprocess_function(examples):
        return tokenizer(examples["text"], padding='max_length', max_length = 512, truncation=True)

In [29]:
#!g1.1
tokenized_CW_train = dataset_train.map(preprocess_function, batched=True)
tokenized_CW_test = dataset_test.map(preprocess_function, batched=True)


Map: 100%|██████████| 8000/8000 [00:02<00:00, 3438.76 examples/s]

Map: 100%|██████████| 2000/2000 [00:00<00:00, 3448.48 examples/s]


In [30]:
#!g1.1
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
#!g1.1
%pip install evaluate

import evaluate

accuracy = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
#!g1.1
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [32]:
#!g1.1
id2label = {0: "0", 1: "1", 2: "2", 3: "3", 4: "4", 5: "5"}
label2id = {"0": 0, "1": 1,"2": 2, "3": 3,"4": 4, "5": 5}

In [33]:
#!g1.1
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", num_labels=6, id2label=id2label, label2id=label2id
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
#!g1.1
training_args = TrainingArguments(
    output_dir="/home/jupyter/datasphere/project/Trained_Bert_classifier_2",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    do_eval=False,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="no",
    report_to='none'
)

In [35]:
#!g1.1
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_CW_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
#!g1.1
trainer.train()

In [22]:
#!g1.1

import transformers
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('/home/jupyter/datasphere/project/Trained_Bert_classifier/checkpoint-500')


In [23]:
#!g1.1
import transformers
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "/home/jupyter/datasphere/project/Trained_Bert_classifier/checkpoint-500", num_labels=8, id2label=id2label, label2id=label2id
) 

In [24]:
#!g1.1
labels = sorted(model.config.label2id, key=model.config.label2id.get)


In [ ]:
#!g1.1
 model.eval()

In [26]:
#!g1.1
print(tokenized_CW_test)

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 485
})


In [27]:
#!g1.1
predictions  = trainer.predict(tokenized_CW_test)

100%|██████████| 61/61 [00:13<00:00,  4.39it/s]


In [28]:
#!g1.1
print(predictions.predictions.shape, predictions.label_ids.shape)

(485, 8) (485,)


In [29]:
#!g1.1
print(predictions)

PredictionOutput(predictions=array([[-0.90430146, -0.695551  ,  6.2991447 , ..., -1.0411364 ,
        -1.1330239 , -0.45338547],
       [-0.15229808, -1.0796051 , -1.2205628 , ...,  6.2253475 ,
        -1.2187221 ,  0.3309724 ],
       [-1.1101298 , -1.0288961 , -0.6363298 , ..., -0.6426865 ,
        -0.6138447 ,  6.612135  ],
       ...,
       [-1.0795137 , -1.0714254 , -0.6115971 , ..., -0.61430794,
        -0.6220971 ,  6.617766  ],
       [-0.95550776, -0.97638416, -1.1292629 , ...,  6.2457085 ,
        -1.2937596 , -0.44293088],
       [-0.99417794, -1.0750242 , -0.73827845, ..., -0.35751623,
        -0.7248068 ,  6.599467  ]], dtype=float32), label_ids=array([2, 7, 7, 2, 2, 5, 7, 1, 7, 3, 5, 6, 7, 2, 4, 1, 2, 1, 2, 1, 4, 3,
       5, 4, 1, 2, 5, 6, 4, 6, 4, 0, 3, 4, 4, 0, 2, 5, 4, 2, 2, 4, 5, 5,
       7, 7, 1, 4, 0, 3, 7, 4, 4, 2, 4, 1, 4, 1, 7, 7, 0, 5, 5, 2, 6, 7,
       7, 0, 2, 3, 0, 0, 7, 4, 7, 3, 7, 7, 3, 7, 4, 5, 3, 7, 6, 5, 7, 7,
       5, 7, 0, 4, 5, 5, 1, 7, 7, 7, 0, 

In [30]:
#!g1.1
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [31]:
#!g1.1
actual = tokenized_CW_test['label']

In [32]:
#!g1.1
from sklearn.metrics import classification_report
    
y_true = actual
y_pred = preds

target_names = ['Bolsheviki', 'Mensheviki', 'Left_SRs', 'Kadets', 'Rights', 'White_army', 'Work_oppose', 'Nobles']

print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

  Bolsheviki       1.00      0.98      0.99        49
  Mensheviki       0.96      0.93      0.94        54
    Left_SRs       0.99      1.00      0.99        66
      Kadets       0.98      1.00      0.99        42
      Rights       1.00      0.98      0.99        65
  White_army       0.93      1.00      0.96        55
 Work_oppose       0.95      0.95      0.95        38
      Nobles       0.99      0.97      0.98       116

    accuracy                           0.98       485
   macro avg       0.97      0.98      0.98       485
weighted avg       0.98      0.98      0.98       485

